Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Select the Python 3.6 kernel 

Note: If you get any issues with Azure AML SDK, please take a look at the [troubleshooting page](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?preserve-view=true&view=azure-ml-py#troubleshooting).

# Install the Azure Machine Learning library

# 加载workspace

In [1]:
# 加载workspace


from azureml.core import Workspace

subscription_id = 'cc80fb14-49de-4506-997b-89f34562676e'
resource_group  = 'shoufei'
workspace_name  = 'xw-ml-ws'

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')

Library configuration succeeded


# 加载模型

In [19]:
import pickle

# Test the model by importing it and providing a sample data point
print("Import the model from model-arm64.pkl")
f2 = open('model.pkl', 'rb')
clf2 = pickle.load(f2)

# Normal (not an anomaly)，预测值返回0
#X_new = [[24.90294136, 1.44463889, 20.89537849, 24]]
#X_new = [[33.40859853, 2.413637808, 20.89162813, 26]]
#X_new = [[34.42109181, 2.528985143, 21.23903786, 25]]

# Anomaly,预测值返回1
X_new = [[33.66995566, 2.44341267, 21.39450979, 26]]
#X_new = [[105.5457931, 10.63179922, 20.62029994, 26]]

print ('New sample: {}'.format(X_new))

pred = clf2.predict(X_new)
print('Predicted class is {}'.format(pred))
print(pred[0])


Import the model from model-arm64.pkl
New sample: [[33.66995566, 2.44341267, 21.39450979, 26]]
Predicted class is [1]
1


# Register Model to cloud

In [20]:
#Register Model to cloud
# 注册完毕以后，在云端https://ml.azure.com/可以看到这个model-arm64.pkl

from azureml.core.model import Model

model = Model.register(model_path = "model.pkl",
                       model_name = "model-demo1.pkl",
                       tags = {'area': "anomaly", 'type': "classification"},
                       description = "Sample anomaly detection model for IOT tutorial",
                       workspace = ws)

Registering model model-demo1.pkl


# Create docker Image

In [21]:
# Create docker Image This specifies the dependencies to include in the environment
# 此处把azureml-sdk修改为azureml-core
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(
    conda_packages=['pandas', 'scikit-learn', 'numpy', 'pip=20.1.1'],
    pip_packages=['azureml-core'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [22]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="iot_score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "iot", 'type': "classification"},
                                 description = "IOT Edge anomaly detection demo")


image = Image.create(name = "tempanomalydetection",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

/tmp/ipykernel_1640/2357192945.py:3: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image_config = ContainerImage.image_configuration(runtime= "python",
/tmp/ipykernel_1640/2357192945.py:10: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  image = Image.create(name = "tempanomalydetection",


Creating image


In [23]:
image.wait_for_creation(show_output = True)

Running...................................................
Succeeded
Image creation operation finished for image tempanomalydetection:3, operation "Succeeded"


# 部署模型

直接docker run模型，映射5001端口
然后使用postman调用
http://127.0.0.1:5001/score 
body当中传递json数据
```
{
    "machine":{
        "temperature":31.16469009,
        "pressure":2.158002669
    },
    "ambient":{
        "temperature":21.17794693,
        "humidity":25
    },
    "timeCreated":"2017-10-27T18:14:02.4911177Z"
}
```